In [1]:
import os
import torch
import random
import datetime
import pandas as pd
import numpy as np

from torch.utils.data import Dataset
from src.datasets import RL4RS, ContentWise, DummyData
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems, get_svd_encoder
from src.embeddings import RecsysEmbedding

experiment_name = 'FlattenedGRU'
device = 'cuda:0'
seed = 123
pkl_path = '../pkl/'

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
torch.__version__

'1.12.1'

# Модель

In [3]:
import torch.nn.functional as F
torch.autograd.set_detect_anomaly(True)

class NeuralClickModel(torch.nn.Module):
    def __init__(self, embedding, output_dim=1, dropout = 0.1):
        super().__init__()
        self.embedding_dim = embedding.embedding_dim
        self.embedding = embedding
        self.rnn_layer = torch.nn.GRU(
            input_size = embedding.embedding_dim, 
            hidden_size = embedding.embedding_dim, 
            batch_first = True,
            dropout=dropout
        )
        self.out_layer = torch.nn.Linear(embedding.embedding_dim, output_dim)


    def forward(self, batch):
        shp = batch['slates_item_indexes'].shape
        item_embs, user_embs = self.embedding(batch)
        item_embs = item_embs.flatten(-3, -2)
        
        # while training, let out model see the future 
        # while testing, it can see only the 
        if self.training:
            indices = (batch['length'] - 1)
        else:
            indices = (batch['in_length'] - 1)
        
        indices[indices<0] = 0
        indices = indices[:, None, None].repeat(1, 1, user_embs.size(-1))
        user_embs = user_embs.gather(1, indices).squeeze(-2).unsqueeze(0)

#         print(indices.shape, user_embs.shape, item_embs.shape, )
        rnn_out, _ = self.rnn_layer(
            item_embs,
            user_embs,
        )
        return self.out_layer(rnn_out).reshape(shp)

# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [4]:
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

model = NeuralClickModel(
    RecsysEmbedding(d.n_items, dummy_matrix, embeddings='neural').to('cpu'),
    output_dim=1
).to('cpu')

train(
    model, 
    dummy_loader, dummy_loader, dummy_loader,
    device=device, lr=1e-3, num_epochs=5000, dummy=True,
    silent=True,
)


biulding affinity matrix...


3it [00:00, 2683.50it/s]
/opt/conda/envs/myenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Test before learning: {'f1': 0.4000000059604645, 'roc-auc': 0.3333333134651184, 'accuracy': 0.25}


train... loss:0.7189432978630066:   0%|                                                                                                    | 1/5000 [00:00<29:11,  2.85it/s]

Val update: epoch: 0 |accuracy: 0.5 | f1: 0.5 | auc: 0.3333333134651184 | treshold: 0.51
Test: accuracy: 0.5 | f1: 0.5 | auc: 0.3333333134651184 | 


train... loss:0.6976741552352905:   0%|                                                                                                    | 3/5000 [00:01<30:55,  2.69it/s]

Val update: epoch: 3 |accuracy: 1.0 | f1: 1.0 | auc: 1.0 | treshold: 0.54
Test: accuracy: 1.0 | f1: 1.0 | auc: 1.0 | 


(NeuralClickModel(
   (embedding): RecsysEmbedding(
     (item_embeddings): Embedding(5, 32)
   )
   (rnn_layer): GRU(32, 32, batch_first=True, dropout=0.1)
   (out_layer): Linear(in_features=32, out_features=1, bias=True)
 ),
 {'f1': 1.0, 'roc-auc': 1.0, 'accuracy': 1.0})

# ContentWise

In [5]:
content_wise_results = []
dataset = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=150)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

20216 data points among 108 batches


In [6]:
for embeddings in ['svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    
    model = NeuralClickModel(
        RecsysEmbedding(train_num_items, train_user_item_matrix, embeddings=embeddings),
        output_dim=1
    ).to(device)

    _, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)


Evaluating FlattenedGRU with svd embeddings


/opt/conda/envs/myenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Test before learning: {'f1': 0.18066997826099396, 'roc-auc': 0.5031406283378601, 'accuracy': 0.10415486246347427}


train... loss:47.45200476050377:   0%|                                                                                                  | 1/5000 [00:50<69:43:38, 50.21s/it]

Val update: epoch: 0 |accuracy: 0.10144636034965515 | f1: 0.18420572578907013 | auc: 0.5617542266845703 | treshold: 0.01
Test: accuracy: 0.09924836456775665 | f1: 0.18057496845722198 | auc: 0.5637004375457764 | 


...

train... loss:29.243009239435196:   0%|▍                                                                                               | 23/5000 [07:15<26:27:20, 19.14s/it]

Val update: epoch: 22 |accuracy: 0.8278579711914062 | f1: 0.3067828118801117 | auc: 0.7333426475524902 | treshold: 0.17
Test: accuracy: 0.8300474286079407 | f1: 0.30520668625831604 | auc: 0.7261786460876465 | 


train... loss:29.048662051558495:   1%|▌                                                                                               | 28/5000 [08:47<25:33:06, 18.50s/it]

Val update: epoch: 27 |accuracy: 0.8572333455085754 | f1: 0.2861778438091278 | auc: 0.7346652746200562 | treshold: 0.15000000000000002
Test: accuracy: 0.8617532253265381 | f1: 0.28954628109931946 | auc: 0.7275290489196777 | 


train... loss:29.155868589878082:   1%|█▎                                                                                              | 68/5000 [21:10<25:35:39, 18.68s/it]


In [7]:
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items

# RL4RS

In [ ]:
rl4rs_results = []
dataset = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=350)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

45942 data points among 106 batches


In [10]:
for embeddings in ['neural', 'svd',  ]:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

    model = NeuralClickModel(
        RecsysEmbedding(train_num_items, train_user_item_matrix, embeddings=embeddings),
        output_dim=1
    ).to(device)


    best_model, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
        silent=True
    )
    
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)


Evaluating FlattenedGRU with neural embeddings
Test before learning: {'f1': 0.15188458561897278, 'roc-auc': 0.5037975907325745, 'accuracy': 0.37591585516929626}


train... loss:64.7874805033207:   0%|                                                                                                   | 1/5000 [00:07<10:38:09,  7.66s/it]

Val update: epoch: 0 |accuracy: 0.7316790223121643 | f1: 0.8233214616775513 | auc: 0.7749164700508118 | treshold: 0.4
Test: accuracy: 0.7273122668266296 | f1: 0.8199799060821533 | auc: 0.7782286405563354 | 


train... loss:54.15752696990967:   0%|                                                                                                  | 2/5000 [00:15<10:35:35,  7.63s/it]

Val update: epoch: 1 |accuracy: 0.7767619490623474 | f1: 0.8362255692481995 | auc: 0.8285561800003052 | treshold: 0.39
Test: accuracy: 0.7797122597694397 | f1: 0.8374955654144287 | auc: 0.8331220149993896 | 


...

train... loss:28.234099194407463:   2%|██▏                                                                                             | 115/5000 [12:24<8:48:29,  6.49s/it]

Val update: epoch: 114 |accuracy: 0.8426207900047302 | f1: 0.8802694082260132 | auc: 0.9133642911911011 | treshold: 0.43
Test: accuracy: 0.841300904750824 | f1: 0.8784427046775818 | auc: 0.9130176305770874 | 


train... loss:27.872017681598663:   2%|██▍                                                                                             | 124/5000 [13:21<8:47:51,  6.50s/it]

Val update: epoch: 123 |accuracy: 0.841943621635437 | f1: 0.8800058960914612 | auc: 0.913579523563385 | treshold: 0.42000000000000004
Test: accuracy: 0.8402369618415833 | f1: 0.8780433535575867 | auc: 0.9129936695098877 | 


train... loss:28.233209624886513:   3%|██▍                                                                                             | 130/5000 [14:05<8:48:12,  6.51s/it]
